In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("The Beatles dataset.csv")

In [3]:
df.head()

,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,title,chords
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01 A Hard Day's Night,N
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01 A Hard Day's Night,N
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01 A Hard Day's Night,N
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01 A Hard Day's Night,N
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01 A Hard Day's Night,N


In [4]:
titles=df.title.unique()

In [33]:
ex_title=titles[104]
print(ex_title)

12 I Want To Tell You


In [34]:
chroma=df[df.title==ex_title].iloc[:,0:12].to_numpy().transpose()
chord_reference_literal=df[df.title==ex_title].chords.to_list()
print("chroma.shape = ",chroma.shape,"chord_reference_literal len = ",len(chord_reference_literal))

chroma.shape =  (12, 1594) chord_reference_literal len =  1594


In [35]:
# DA SISTEMARE
def omologous(root):
    flats=["Cb","Db","Eb","Fb","Gb","Ab","Bb"]
    omo=["B","C#","D#","E","F#","G#","A#"]
    if root in flats:
        return omo[flats.index(root)]
    else:
        return root
    
def chord_to_triad(lab):
    triads_labs=["C","C#","D","D#","E","F","F#","G","G#","A","A#","B","C:min","C#:min","D:min","D#:min","E:min","F:min","F#:min","G:min","G#:min","A:min","A#:min","B:min"]
    separated_lab=lab.split(":")
    n=len(separated_lab)
    if n==1:
        return omologous(lab.split("/")[0])
    else:
        root=separated_lab[0]
        if separated_lab[1][0].isdigit():
            return omologous(root.split("/")[0])
        elif separated_lab[1][:3]=="min" or separated_lab[1][:3]=="dim":
            return omologous(root.split("/")[0])+":min"
        else:
            return omologous(root.split("/")[0])
  

In [36]:
chord_reference_literal_reduced=[]
for ii in range(len(chord_reference_literal)):
    chord_reference_literal_reduced.append(chord_to_triad(chord_reference_literal[ii]))

In [37]:
possible=["N","C","C#","D","D#","E","F","F#","G","G#","A","A#","B","C:min","C#:min","D:min","D#:min","E:min","F:min","F#:min","G:min","G#:min","A:min","A#:min","B:min"]
print([x for x in chord_reference_literal_reduced if x not in possible])
print(set(chord_reference_literal_reduced)-set(["N","C","C#","D","D#","E","F","F#","G","G#","A","A#","B","C:min","C#:min","D:min","D#:min","E:min","F:min","F#:min","G:min","G#:min","A:min","A#:min","B:min"]))

[]
set()


In [42]:
# CHecking for all
df=pd.read_csv("The Beatles dataset.csv")
all_chords=df.chords.to_list()
all_reduced=[]
for ii in range(len(all_chords)):
    all_reduced.append(chord_to_triad(all_chords[ii]))

In [43]:
possible=["N","C","C#","D","D#","E","F","F#","G","G#","A","A#","B","C:min","C#:min","D:min","D#:min","E:min","F:min","F#:min","G:min","G#:min","A:min","A#:min","B:min"]
print([x for x in all_reduced if x not in possible])
print(set(all_reduced)-set(possible))

[]
set()
